# Rent Trend Line Chart

Shows 5-year rental price trends across Toronto neighbourhoods.

## 1. Data Reference

### Source Tables

| Table | Grain | Key Columns |
|-------|-------|-------------|
| `mart_neighbourhood_housing` | neighbourhood × year | year, avg_rent_2bed, rent_yoy_change_pct |

### SQL Query

In [1]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Load .env from project root
load_dotenv("../../.env")

engine = create_engine(os.environ["DATABASE_URL"])

# City-wide average rent by year
query = """
SELECT
    year,
    AVG(avg_rent_bachelor) as avg_rent_bachelor,
    AVG(avg_rent_1bed) as avg_rent_1bed,
    AVG(avg_rent_2bed) as avg_rent_2bed,
    AVG(avg_rent_3bed) as avg_rent_3bed,
    AVG(rent_yoy_change_pct) as avg_yoy_change
FROM mart_toronto.mart_neighbourhood_housing
WHERE year >= (SELECT MAX(year) - 5 FROM mart_toronto.mart_neighbourhood_housing)
GROUP BY year
ORDER BY year
"""

df = pd.read_sql(query, engine)
print(f"Loaded {len(df)} years of rent data")

Loaded 6 years of rent data


### Transformation Steps

1. Aggregate rent by year (city-wide average)
2. Convert year to datetime for proper x-axis
3. Reshape for multi-line chart by bedroom type

In [2]:
# Create date column from year
df["date"] = pd.to_datetime(df["year"].astype(str) + "-01-01")

# Melt for multi-line chart
df_melted = df.melt(
    id_vars=["year", "date"],
    value_vars=["avg_rent_bachelor", "avg_rent_1bed", "avg_rent_2bed", "avg_rent_3bed"],
    var_name="bedroom_type",
    value_name="avg_rent",
)

# Clean labels
df_melted["bedroom_type"] = df_melted["bedroom_type"].map(
    {
        "avg_rent_bachelor": "Bachelor",
        "avg_rent_1bed": "1 Bedroom",
        "avg_rent_2bed": "2 Bedroom",
        "avg_rent_3bed": "3 Bedroom",
    }
)

### Sample Output

In [3]:
df[
    [
        "year",
        "avg_rent_bachelor",
        "avg_rent_1bed",
        "avg_rent_2bed",
        "avg_rent_3bed",
        "avg_yoy_change",
    ]
]

,year,avg_rent_bachelor,avg_rent_1bed,avg_rent_2bed,avg_rent_3bed,avg_yoy_change
0,2020,1206.0,1424.0,1636.0,1834.0,4.74
1,2021,1217.0,1441.0,1681.0,1891.0,2.75
2,2022,1311.0,1529.0,1780.0,2034.0,5.89
3,2023,1415.0,1691.0,1971.0,2203.0,10.73
4,2024,1448.0,1714.0,1979.0,2221.0,0.41
5,2025,1494.0,1761.0,2056.0,2319.0,3.89


## 2. Data Visualization

### Figure Factory

Uses `create_price_time_series` from `portfolio_app.figures.toronto.time_series`.

**Key Parameters:**
- `date_column`: 'date'
- `price_column`: 'avg_rent'
- `group_column`: 'bedroom_type' (for multi-line)

In [4]:
import sys

sys.path.insert(0, "../..")

from portfolio_app.figures.toronto.time_series import create_price_time_series

data = df_melted.to_dict("records")

fig = create_price_time_series(
    data=data,
    date_column="date",
    price_column="avg_rent",
    group_column="bedroom_type",
    title="Toronto Average Rent Trend (5 Years)",
)

fig.show()

### YoY Change Analysis

In [5]:
# Show year-over-year changes
print("Year-over-Year Rent Change (%)")
df[["year", "avg_yoy_change"]].dropna()

Year-over-Year Rent Change (%)


,year,avg_yoy_change
0,2020,4.74
1,2021,2.75
2,2022,5.89
3,2023,10.73
4,2024,0.41
5,2025,3.89
